In [42]:
from rdkit import Chem
from rdkit.Chem import AllChem
from pathlib import Path
import os

In [43]:
def CalculateSpinMultiplicity(Mol):
 """Calculate spin multiplicity of a molecule. The spin multiplicity is calculated
 from the number of free radical electrons using Hund's rule of maximum
 multiplicity defined as 2S + 1 where S is the total electron spin. The
 total spin is 1/2 the number of free radical electrons in a molecule.

 Arguments:
 Mol (object): RDKit molecule object.

 Returns:
 int : Spin multiplicity.

 """

 # Calculate spin multiplicity using Hund's rule of maximum multiplicity...
 NumRadicalElectrons = 0
 for Atom in Mol.GetAtoms():
    NumRadicalElectrons += Atom.GetNumRadicalElectrons()

 TotalElectronicSpin = NumRadicalElectrons/2
 SpinMultiplicity = 2 * TotalElectronicSpin + 1

 return int(SpinMultiplicity)

In [44]:
def get_method(multiplicity):
    if multiplicity % 2 == 0 :
        method = 'ROHF'
    else:
        method = 'RHF'
    return method

In [48]:
molfile = Chem.SDMolSupplier("../data/sample.sdf")

for mol in molfile:
    mol = Chem.AddHs(mol)
    chem_class = mol.GetProp("Class")
    inchikey = mol.GetProp("InChIKey")
    name = mol.GetProp("NAME")

    moldir = Path.cwd() / "classes" / chem_class / inchikey / "Optimization"
    Path.mkdir(moldir, parents=True, exist_ok=True)
    mol_path = moldir  / (inchikey + ".inp")
    Path.touch(mol_path)

    AllChem.EmbedMolecule(mol, maxAttempts=10000,useRandomCoords=False)
    conf = mol.GetConformer()
    multiplicity = CalculateSpinMultiplicity(mol)
    opt = f''' $CONTRL SCFTYP={get_method(multiplicity)} MULT={multiplicity} NPRINT=-5 RUNTYP=OPTIMIZE $END\n $STATPT OPTTOL=0.0005 NSTEP=100 NPRT=-2 $END\n $BASIS  GBASIS=N31 NGAUSS=6 $END'''

    with open(mol_path, 'w') as outfile:
        outfile.write(f"{opt}{os.linesep}")
        outfile.write(f"{os.linesep}")
        outfile.write(f" $DATA{os.linesep}")
        outfile.write(f"{name}{os.linesep}")
        outfile.write(f"C1{os.linesep}")
        
        for (i, a) in enumerate(mol.GetAtoms()):
            pos = conf.GetAtomPosition(i)
            outfile.write(f"{a.GetSymbol()}\t{a.GetAtomicNum()}\t{pos.x}\t{pos.y}\t{pos.z}{os.linesep}")
        
        outfile.write(f"$END{os.linesep}")


SyntaxError: invalid decimal literal (887470282.py, line 28)